In [1]:
import warnings
warnings.filterwarnings('ignore')

import AutoML
import pandas as pd
import numpy as np
from importlib import reload
pd.set_option('display.max_columns', None)
reload(AutoML)
import matplotlib.pyplot as plt
%matplotlib inline

/Users/zihaoguo/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
data = pd.read_csv('mylsn_raw.csv')
data = data.drop(['Unnamed: 0'], axis=1)
data.head()

,application_id,school,status,sentmonth,decisionmonth,app_id,sent,decision,url,money,lsat,gpa,urm,ed,feewaiver,complete_ts,decision_ts,aa,attending,cycle,state,race,sex,yearsout,schooltype,major,nontrad,international,lgbt,tfa,military,lsat1,lsat2,lsat3
0,2185601,gulc,Ac,11,12,187539,11/07/05,12/01/05,http://lawschoolnumbers.com/5,0,173,3.74,0,0,0,1131944400,1133413200,0,0,506,New York,NaN,Female,NaN,NaN,NaN,0,0,0,0,0,173.0,0,0
1,2185600,uva,Ac,10,12,187539,10/12/05,12/07/05,http://lawschoolnumbers.com/5,0,173,3.74,0,0,0,1130734800,1133931600,0,0,506,New York,NaN,Female,NaN,NaN,NaN,0,0,0,0,0,173.0,0,0
2,2185599,penn,Ac,11,12,187539,11/01/05,12/16/05,http://lawschoolnumbers.com/5,0,173,3.74,0,0,0,1131944400,1134709200,0,0,506,New York,NaN,Female,NaN,NaN,NaN,0,0,0,0,0,173.0,0,0
3,2185598,nyu,Ac,10,0,187539,10/12/05,--,http://lawschoolnumbers.com/5,0,173,3.74,0,0,0,1130734800,0,0,0,506,New York,NaN,Female,NaN,NaN,NaN,0,0,0,0,0,173.0,0,0
4,2185597,chicago,Ac,11,2,187539,11/08/05,02/23/06,http://lawschoolnumbers.com/5,0,173,3.74,0,0,0,1133413200,1140670800,0,0,506,New York,NaN,Female,NaN,NaN,NaN,0,0,0,0,0,173.0,0,0


In [10]:
df = data.copy()

In [12]:
train = df.sample(frac=0.5, random_state=200)
test=df.drop(train.index)

In [17]:
# set up the parameters for StandardDataset class
input_columns = ['sex', 'gpa', 'lsat', 'lsat1']#,'school']  # school will generate too many columns
label_name = 'status'
favorable_classes = ['Ac']
protected_attribute_names = ['sex']
privileged_classes = [['female','female']]
privileged_groups = [{'sex':1}]
unprivileged_groups = [{'sex': 0}]
categorical_features = []#['school']
features_to_keep = ['sex', 'gpa', 'lsat']#,'school']
features_to_drop = []
is_valid = False

In [18]:
model = AutoML.FairAutoML(input_columns, label_name, favorable_classes, 
                 protected_attribute_names, privileged_classes,
                 privileged_groups, unprivileged_groups,
                 categorical_features, features_to_keep, 
                 features_to_drop, is_valid)

## dataset_metric you can select and their ranges
'mean_difference': [-1,1] <br>
'statistical_parity_difference': [-1,1] <br>
'disparate_impact': > 0 <br>
'consistency': [0, 1]

## classifier_metric
"Statistical parity difference" : [-0.1, 0.1]<br>
"Mean difference": [-0.1, 0.1] <br>
"Disparate impact": [0.8,1.2] <br>
"Average odds difference" [-0.1, 0.1] <br>
"Equal opportunity difference" [-0.1, 0.1] <br>
"Theil index": [0, 0.2]

In [ ]:
model.fit(train, dataset_metric='mean_difference', dataset_metric_threshold=0.001,
            classifier_metric='Equal opportunity difference', optim_options = None,
            time_left_for_this_task=200, per_run_time_limit=20, train_split_size=0.8,
            verbose=True)

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.056890
No discrimination potential, no need for fairness preprocessing:  False
--------------Start Preprocessing--------------
Difference in mean outcomes between unprivileged and privileged groups after Disparate Impact remover with repair_level = 0.1 = -0.05688974182075057
--------------Start Fitting Model--------------
[WARNING] [2019-12-07 20:01:19,836:EnsembleBuilder(1):84fdcb8acda0ea05954d39762b403cde] No models better than random - using Dummy Score!
[WARNING] [2019-12-07 20:01:19,858:EnsembleBuilder(1):84fdcb8acda0ea05954d39762b403cde] No models better than random - using Dummy Score!
[WARNING] [2019-12-07 20:01:21,864:EnsembleBuilder(1):84fdcb8acda0ea05954d39762b403cde] No models better than random - using Dummy Score!
[WARNING] [2019-12-07 20:01:23,871:EnsembleBuilder(1):84fdcb8acda0ea05954d39762b403cde] No models better than random - using Dummy Score!
[WARNING] [2019-12-07 20:01:25,877:EnsembleBuild

In [10]:
# Evaluate model performance on Test
metrics = model.evaluate(test)

[WARNING] [2019-12-07 19:23:46,835:root] Missing Data: 74910 rows removed from StandardDataset.
Data has been transformed into standardized dataframe.
Balanced accuracy = 0.5392
Statistical parity difference = 0.0284
Mean difference = 0.0284
Disparate impact = 1.2859
Average odds difference = 0.0280
Equal opportunity difference = 0.0393
Theil index = 0.4514
Optimal classification threshold (after fairness processing) = 0.4900


,Balanced accuracy,Statistical parity difference,Mean difference,Disparate impact,Average odds difference,Equal opportunity difference,Theil index
0,0.5392,0.028358,0.028358,1.285917,0.028,0.039275,0.451393


In [11]:
pred_labels = model.predict(test)

[WARNING] [2019-12-07 19:23:59,005:root] Missing Data: 74910 rows removed from StandardDataset.
Data has been transformed into standardized dataframe.
